In [1]:
# Imports 
import os
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
import time
import warnings
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import ExtraTreesRegressor
import xgboost as xgb
from bayes_opt import BayesianOptimization
import scipy.interpolate

# Defines
SEED = 2017
N_JOBS = 18
EMISSIONS = "Counts.Particles.PNC3"
pd.set_option('display.float_format', lambda x: '%.3f' % x)
%matplotlib inline
plt.rcParams["patch.force_edgecolor"] = True
warnings.filterwarnings("ignore")

MODEL = "xg"

/home/ubuntu/anaconda2/envs/jcs_dl/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Fit, predict and return error
def run_et(max_features, max_depth, min_samples_split, min_samples_leaf) :
    # Fit model
    et = ExtraTreesRegressor(n_estimators = 100, 
                             max_features = max_features, 
                             max_depth = int(max_depth), 
                             min_samples_split = int(min_samples_split), 
                             min_samples_leaf = int(min_samples_leaf), 
                             n_jobs = -1, 
                             random_state = SEED)
    et.fit(X_train, y_train)
    
    # Make predictions
    preds = et.predict(X_test) 

    # Compute error
    rmse = np.sqrt(mean_squared_error(y_test, preds))

    return((-1) * rmse)


def run_xgb(max_depth, min_child_weight, subsample, colsample_bytree, colsample_bylevel) :

    def xgb_mse(preds, dtrain) :
        labels = dtrain.get_label()
        return ("mse", mean_squared_error(preds, labels))

    xg_params = {"nthread" : N_JOBS, 
                 "eta" : 0.3, 
                 "max_depth" : int(max_depth), 
                 "min_child_weight" : int(min_child_weight), 
                 "subsample" : subsample, 
                 "colsample_bytree" : colsample_bytree, 
                 "colsample_bylevel" : colsample_bylevel}    
    
    xg_trainVal = xgb.DMatrix(X_trainVal, label = y_trainVal)
    xg_testVal = xgb.DMatrix(X_testVal, label = y_testVal)
    xg_train = xgb.DMatrix(X_train, label = y_train)
    
    watchlist = [(xg_trainVal, "train"), (xg_testVal, "eval")]    
    
    # Fit model using validation set
    xg = xgb.train(
        params = xg_params,
        dtrain = xg_trainVal,
        evals = watchlist, 
        num_boost_round = 100000,
        early_stopping_rounds = 50,
        feval = xgb_mse, 
        maximize = False,
        verbose_eval = False)
    
    # Now fit on whole training set
    xg2 = xgb.train(
        params = xg_params, 
        dtrain = xg_train, 
        num_boost_round = int(xg.best_ntree_limit / 0.67),
        feval = xgb_mse, 
        maximize = False,
        verbose_eval = False)
    
    # Make predictions
    xg_test = xgb.DMatrix(X_test)
    preds = xg2.predict(xg_test) 

    # Compute error
    rmse = np.sqrt(mean_squared_error(y_test, preds))

    return((-1) * rmse)


In [3]:
# Init Bayesian Optimization
et_max_features_min = 0.4
et_max_features_max = 1.0
et_max_depth_min = 3
et_max_depth_max = 6
et_min_samples_split_min = 2
et_min_samples_split_max = 20
et_min_samples_leaf_min = 2
et_min_samples_leaf_max = 20

xgb_max_depth_min = 5
xgb_max_depth_max = 12
xgb_min_child_weight_min = 10
xgb_min_child_weight_max = 25
xgb_subsample_min = 0.4
xgb_subsample_max = 0.9
xgb_colsample_bytree_min = 0.4
xgb_colsample_bytree_max = 1.0
xgb_colsample_bylevel_min = 0.3
xgb_colsample_bylevel_max = 1.0


def initBO() :
    if (MODEL == "et") :
        # Define the features to explore
        bo = BayesianOptimization(run_et, {
            "max_features" : (et_max_features_min, et_max_features_max), 
            "max_depth" : (et_max_depth_min, et_max_depth_max),
            "min_samples_split" : (et_min_samples_split_min, et_min_samples_split_max),
            "min_samples_leaf" : (et_min_samples_leaf_min, et_min_samples_leaf_max),
        })
        
        # Define the initial exploration : 10 points approximately covering the range of each parameter
        bo.explore({
            "max_features" : [0.6, 0.5, 0.4, 0.9, 0.7, 0.5, 0.6, 0.8, 1.0, 0.7],
            "max_depth" : [6, 3, 4, 5, 3, 6, 4, 5, 3, 6], 
            "min_samples_split" : [5, 15, 20, 10, 20, 5, 15, 2, 10, 20],
            "min_samples_leaf" : [5, 10, 20, 15, 10, 20, 10, 15, 2, 10]
        })
    elif (MODEL == "xg") :
        # Define the features to explore
        bo = BayesianOptimization(run_xgb, {
            "max_depth" : (xgb_max_depth_min, xgb_max_depth_max), 
            "min_child_weight" : (xgb_min_child_weight_min, xgb_min_child_weight_max),
            "subsample" : (xgb_subsample_min, xgb_subsample_max),
            "colsample_bytree" : (xgb_colsample_bytree_min, xgb_colsample_bytree_max),
            "colsample_bylevel" : (xgb_colsample_bylevel_min, xgb_colsample_bylevel_max), 
        })
        
        # Define the initial exploration : 10 points approximately covering the range of each parameter
        bo.explore({
            "max_depth" : [8, 6, 7, 12, 6, 5, 7, 8, 10, 9],
            "min_child_weight" : [14, 10, 25, 18, 10, 15, 25, 12, 17, 22],
            "subsample" : [0.6, 0.9, 0.7, 0.5, 0.6, 0.4, 0.7, 0.9, 0.5, 0.8],
            "colsample_bytree" : [0.4, 0.7, 1.0, 0.6, 0.9, 0.5, 0.8, 0.6, 0.8, 0.9], 
            "colsample_bylevel" :  [0.8, 0.6, 0.7, 1.0, 0.3, 0.9, 1.0, 0.5, 0.7, 0.4],
            })
    
    return(bo)


In [4]:
# Plot results of optimization
def graphBO(history_df, param1, param2, param1min, param1max, param2min, param2max, ratio) :
    x, y, z = history_df[param1].values, history_df[param2].values, history_df["RMSE"].values

    # Set up a regular grid of interpolation points
    xi, yi = np.linspace(param1min, param1max, 100), np.linspace(param2min, param2max, 100)
    xi, yi = np.meshgrid(xi, yi)

    # Interpolate
    rbf = scipy.interpolate.Rbf(x, y, z, function = "multiquadric", smooth = 0.5)
    zi = rbf(xi, yi)

    plt.figure()
    plt.imshow(zi, 
               cmap = "plasma", 
               aspect = ratio,
               vmin = z.min(), 
               vmax = z.max(), 
               origin = "lower",
               extent = [param1min, param1max, param2min, param2max])
    q = plt.scatter(x, y, c = z, cmap = "plasma")
    plt.colorbar(q)
    plt.xlabel(param1)
    plt.ylabel(param2)
    #plt.savefig("figures/XGB_" + param1 + "_" + param2 + ".png")
    plt.show(block = False)

<b>Script</b>

In [5]:
# Load data
dfs = []
i = 0
for name in os.listdir("../../../data/temp_data/RDE") :
    df = pd.read_csv("../../../data/temp_data/RDE/df" + str(i) + ".csv")
    print(df.shape)
    dfs.append(df)
    i += 1


(8723, 159)
(8781, 159)
(8722, 159)
(8732, 159)
(8789, 159)
(8790, 159)
(8789, 159)
(8731, 159)
(8787, 159)


In [ ]:
# Define train and test sets for BO process
X_train = pd.concat([dfs[0], dfs[1], dfs[2]], axis = 0)
y_train = X_train["target25"]
print(X_train.shape)
print(y_train.shape)

X_test = dfs[4]
y_test = X_test["target25"]
print(X_test.shape)
print(y_test.shape)

# Create validation set needed for some models
if (MODEL != "et") :
    X_trainVal = pd.concat([dfs[0], dfs[1]], axis = 0)
    y_trainVal = X_trainVal["target25"]
    print(X_trainVal.shape)
    print(y_trainVal.shape)

    X_testVal = dfs[2]
    y_testVal = X_testVal["target25"]
    print(X_testVal.shape)
    print(y_testVal.shape)

cols = [c for c in X_train.columns if c[:6] == "target"]
X_train.drop(cols, axis = 1, inplace = True)
#X_trainVal.drop(cols, axis = 1, inplace = True)
X_test.drop(cols, axis = 1, inplace = True)
#X_testVal.drop(cols, axis = 1, inplace = True)


(26226, 159)
(26226,)
(8789, 159)
(8789,)
(17504, 159)
(17504,)
(8722, 159)
(8722,)


In [ ]:
# Run optimization
start = time.time()
bo = initBO()
bo.maximize(init_points = 10, n_iter = 110, xi = 0.05)
print("BayesianOptimization took %.2f seconds" % ((time.time() - start)))

Initialization
-------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bylevel |   colsample_bytree |   max_depth |   min_child_weight |   subsample | 
    1 | 06m27s | -370.78274 |              0.8000 |             0.4000 |      8.0000 |            14.0000 |      0.6000 | 
    2 | 00m05s | -339.65701 |              0.6000 |             0.7000 |      6.0000 |            10.0000 |      0.9000 | 
    3 | 00m01s | -318.83507 |              0.7000 |             1.0000 |      7.0000 |            25.0000 |      0.7000 | 
    4 | 00m11s | -375.76114 |              1.0000 |             0.6000 |     12.0000 |            18.0000 |      0.5000 | 
    5 | 00m05s | -390.10962 |              0.3000 |             0.9000 |      6.0000 |            10.0000 |      0.6000 | 
    6 | 00m04s | -416.40764 |              0.9000 |             0.5000 |      5.0000 |            15.0000 |      0.4000 | 
  

In [ ]:
# Store and look at results
history_df = pd.DataFrame(bo.res["all"]["params"])
history_df2 = pd.DataFrame(bo.res["all"]["values"])
history_df = pd.concat((history_df, history_df2), axis = 1)
history_df.rename(columns = { 0 : "RMSE"}, inplace = True)
history_df.index.names = ["Iteration"]
history_df.sort_values(["RMSE"], ascending = False, inplace = True)
#history_df.to_csv("../../data/temp_results/bo.csv")
display(history_df.head(10))


In [ ]:
if (MODEL == "et") :
    graphBO(history_df, "max_features", "max_depth", et_max_features_min, et_max_features_max, et_max_depth_min, et_max_depth_max, 0.1)
    graphBO(history_df, "max_features", "min_samples_split", et_max_features_min, et_max_features_max, et_min_samples_split_min, et_min_samples_split_max, 0.02)
    graphBO(history_df, "max_features", "min_samples_leaf", et_max_features_min, et_max_features_max, et_min_samples_leaf_min, et_min_samples_leaf_max, 0.02)
    graphBO(history_df, "max_depth", "min_samples_split", et_max_depth_min, et_max_depth_max, et_min_samples_split_min, et_min_samples_split_max, 0.15)
    graphBO(history_df, "max_depth", "min_samples_leaf", et_max_depth_min, et_max_depth_max, et_min_samples_leaf_min, et_min_samples_leaf_max, 0.15)
    graphBO(history_df, "min_samples_split", "min_samples_leaf", et_min_samples_split_min, et_min_samples_split_max, et_min_samples_leaf_min, et_min_samples_leaf_max, 0.75)
elif (MODEL == "xg") :
    graphBO(history_df, "max_depth", "min_child_weight", xgb_max_depth_min, xgb_max_depth_max, xgb_min_child_weight_min, xgb_min_child_weight_max, 0.35)
    graphBO(history_df, "max_depth", "subsample", xgb_max_depth_min, xgb_max_depth_max, xgb_subsample_min, xgb_subsample_max, 10)
    graphBO(history_df, "max_depth", "colsample_bytree", xgb_max_depth_min, xgb_max_depth_max, xgb_colsample_bytree_min, xgb_colsample_bytree_max, 8)
    graphBO(history_df, "max_depth", "colsample_bylevel", xgb_max_depth_min, xgb_max_depth_max, xgb_colsample_bylevel_min, xgb_colsample_bylevel_max, 8)
    graphBO(history_df, "min_child_weight", "subsample", xgb_min_child_weight_min, xgb_min_child_weight_max, xgb_subsample_min, xgb_subsample_max, 20)
    graphBO(history_df, "min_child_weight", "colsample_bytree", xgb_min_child_weight_min, xgb_min_child_weight_max, xgb_colsample_bytree_min, xgb_colsample_bytree_max, 18)
    graphBO(history_df, "min_child_weight", "colsample_bylevel", xgb_min_child_weight_min, xgb_min_child_weight_max, xgb_colsample_bylevel_min, xgb_colsample_bylevel_max, 18)
    graphBO(history_df, "subsample", "colsample_bytree", xgb_subsample_min, xgb_subsample_max, xgb_colsample_bytree_min, xgb_colsample_bytree_max, 0.8)
    graphBO(history_df, "subsample", "colsample_bylevel", xgb_subsample_min, xgb_subsample_max, xgb_colsample_bylevel_min, xgb_colsample_bylevel_max, 0.8)
    graphBO(history_df, "colsample_bytree", "colsample_bylevel", xgb_colsample_bytree_min, xgb_colsample_bytree_max, xgb_colsample_bylevel_min, xgb_colsample_bylevel_max, 1)
else :
    graphBO(history_df, "max_depth", "num_leaves", max_depth_min, max_depth_max, num_leaves_min, num_leaves_max, 0.2)
    graphBO(history_df, "max_depth", "min_data_in_leaf", max_depth_min, max_depth_max, min_data_in_leaf_min, min_data_in_leaf_max, 0.2)
    graphBO(history_df, "max_depth", "feature_fraction", max_depth_min, max_depth_max, feature_fraction_min, feature_fraction_max, 8)
    graphBO(history_df, "max_depth", "bagging_fraction", max_depth_min, max_depth_max, bagging_fraction_min, bagging_fraction_max, 8)
    graphBO(history_df, "max_depth", "bagging_freq", max_depth_min, max_depth_max, bagging_freq_min, bagging_freq_max, 0.1)
    graphBO(history_df, "num_leaves", "min_data_in_leaf", num_leaves_min, num_leaves_max, min_data_in_leaf_min, min_data_in_leaf_max, 1)
    graphBO(history_df, "num_leaves", "feature_fraction", num_leaves_min, num_leaves_max, feature_fraction_min, feature_fraction_max, 25)
    graphBO(history_df, "num_leaves", "bagging_fraction", num_leaves_min, num_leaves_max, bagging_fraction_min, bagging_fraction_max, 25)
    graphBO(history_df, "num_leaves", "bagging_freq", num_leaves_min, num_leaves_max, bagging_freq_min, bagging_freq_max, 0.4)
    graphBO(history_df, "min_data_in_leaf", "feature_fraction", min_data_in_leaf_min, min_data_in_leaf_max, feature_fraction_min, feature_fraction_max, 25)
    graphBO(history_df, "min_data_in_leaf", "bagging_fraction", min_data_in_leaf_min, min_data_in_leaf_max, bagging_fraction_min, bagging_fraction_max, 25)
    graphBO(history_df, "min_data_in_leaf", "bagging_freq", min_data_in_leaf_min, min_data_in_leaf_max, bagging_freq_min, bagging_freq_max, 0.5)
    graphBO(history_df, "feature_fraction", "bagging_fraction", feature_fraction_min, feature_fraction_max, bagging_fraction_min, bagging_fraction_max, 1)
    graphBO(history_df, "feature_fraction", "bagging_freq", feature_fraction_min, feature_fraction_max, bagging_freq_min, bagging_freq_max, 0.01)
    graphBO(history_df, "bagging_fraction", "bagging_freq", bagging_fraction_min, bagging_fraction_max, bagging_freq_min, bagging_freq_max, 0.01)
